# Notebook to evaluate a trained model with pyfolio
Note: pyfolio must be install using pip install git+https://github.com/quantopian/pyfolio. There are known bugs with pip install pyfolio.

In [20]:
import os
import datetime

import pandas as pd
import matplotlib
import pyfolio

from deep_rl_asset_allocation.configs import paths_config, data_config
from deep_rl_asset_allocation.utils import data_loader_utils

In [21]:
# "Agg" backend is for writing to file, not for rendering in a window
matplotlib.use('Agg')

# plotting in notebook
%matplotlib inline

### Helper Functions

In [22]:
def get_daily_return_as_df(df):
    df['daily_return'] = df.account_value.pct_change(1)
    sharpe_ratio = (252**0.5) * df['daily_return'].mean() / df['daily_return'].std()
    print(f'Sharpe Ratio: {sharpe_ratio*100:.2f} %')
    return df


def get_daily_return_as_series(df):
    df1 = df.copy()
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1.set_index('Date', drop=False, inplace=True)
    df1.index = df1.index.tz_localize('UTC')
    del df1['Date']
    ts = pd.Series(df1['daily_return'].values, index=df1.index)
    return ts


def get_training_results_account_value(df_unique_trade_dates):
    df_account_value = pd.DataFrame()
    for training_iteration, idx in enumerate(range(rebalance_window + validation_window, len(unique_trade_dates), rebalance_window)):
        asset_memory_csv_filename = os.path.join(paths_config.results_csv_dir, f'total_asset_value_test_{training_iteration}.csv')
        df = pd.read_csv(asset_memory_csv_filename)
        df_account_value = df_account_value.append(df, ignore_index=True)
    # change the column name
    df_account_value = pd.DataFrame({'account_value': df_account_value['0']})
    # get sharpe ratio
    sharpe_ratio = (252**0.5) * df_account_value.account_value.pct_change(1).mean() / df_account_value.account_value.pct_change(1).std()
    print(f'Sharpe Ratio: {sharpe_ratio*100:.2f} %')
    # merge with unique trade dates from 2016-01-04 or index 64
    df_account_value = df_account_value.join(df_unique_trade_dates.reset_index(drop=True))
    df_account_value = df_account_value[['date', 'account_value']]
    return df_account_value

### DJIA: Out-of-sample Test Data

In [23]:
# load DJIA - out of sample data (test data)
df_djia_out_of_sample = pd.read_csv(paths_config.TESTING_DATA_FILE)

df_djia_out_of_sample

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-02,8772.250000,9065.280273,8760.780273,9034.690430,9034.690430,213700000
1,2009-01-05,9027.129883,9034.370117,8892.360352,8952.889648,8952.889648,233760000
2,2009-01-06,8954.570313,9088.059570,8940.950195,9015.099609,9015.099609,215410000
3,2009-01-07,8996.940430,8996.940430,8719.919922,8769.700195,8769.700195,266710000
4,2009-01-08,8769.940430,8770.019531,8651.190430,8742.459961,8742.459961,226620000
...,...,...,...,...,...,...,...
2923,2020-08-13,27922.509766,27986.099609,27789.779297,27896.720703,27896.720703,360160000
2924,2020-08-14,27828.929688,27977.810547,27759.390625,27931.019531,27931.019531,275540000
2925,2020-08-17,27970.050781,27999.810547,27816.400391,27844.910156,27844.910156,286860000
2926,2020-08-18,27853.480469,27891.119141,27668.789063,27778.070313,27778.070313,276170000


In [24]:
# get data between dates
df_djia_out_of_sample = df_djia_out_of_sample[(df_djia_out_of_sample['Date'] >= str(data_config.TESTING_START)) & (df_djia_out_of_sample['Date'] <= str(data_config.TESTING_END))]
df_djia_out_of_sample = df_djia_out_of_sample.reset_index(drop=True)

# get daily return as relative difference in adjusted close price of DJIA
df_djia_out_of_sample['daily_return'] = df_djia_out_of_sample['Adj Close'].pct_change(1)

df_djia_out_of_sample

,Date,Open,High,Low,Close,Adj Close,Volume,daily_return
0,2016-01-04,17405.480469,17405.480469,16957.630859,17148.939453,17148.939453,148060000,NaN
1,2016-01-05,17147.500000,17195.839844,17038.609375,17158.660156,17158.660156,105750000,0.000567
2,2016-01-06,17154.830078,17154.830078,16817.619141,16906.509766,16906.509766,120250000,-0.014695
3,2016-01-07,16888.359375,16888.359375,16463.630859,16514.099609,16514.099609,176240000,-0.023211
4,2016-01-08,16519.169922,16651.890625,16314.570313,16346.450195,16346.450195,141850000,-0.010152
...,...,...,...,...,...,...,...,...
1090,2020-05-04,23581.550781,23769.560547,23361.160156,23749.759766,23749.759766,355360000,0.001099
1091,2020-05-05,23958.880859,24169.720703,23868.910156,23883.089844,23883.089844,369710000,0.005614
1092,2020-05-06,23978.880859,24054.589844,23661.140625,23664.640625,23664.640625,377450000,-0.009147
1093,2020-05-07,23837.210938,24094.619141,23834.390625,23875.890625,23875.890625,369890000,0.008927


In [25]:
series_djia_out_of_sample = get_daily_return_as_series(df_djia_out_of_sample)
series_djia_out_of_sample

Date
2016-01-04 00:00:00+00:00         NaN
2016-01-05 00:00:00+00:00    0.000567
2016-01-06 00:00:00+00:00   -0.014695
2016-01-07 00:00:00+00:00   -0.023211
2016-01-08 00:00:00+00:00   -0.010152
                               ...   
2020-05-04 00:00:00+00:00    0.001099
2020-05-05 00:00:00+00:00    0.005614
2020-05-06 00:00:00+00:00   -0.009147
2020-05-07 00:00:00+00:00    0.008927
2020-05-08 00:00:00+00:00    0.019075
Length: 1095, dtype: float64

### Load Training Results: Account Value

In [26]:
# load Dow 30 - in sample data (training data)
df_djia_in_sample = data_loader_utils.load_preprocessed_djia_data()

# rebalance_window is the number of months to retrain the model
rebalance_window = data_config.REBALANCE_WINDOW
# validation_window is the number of months to validation the model and select for trading
validation_window = data_config.VALIDATION_WINDOW

# get unique trade dates to load model results
unique_trade_dates = data_loader_utils.get_data_between_dates(df_djia_in_sample, start=data_config.VALIDATION_START, end=data_config.TESTING_END).date.unique()
df_unique_trade_dates = pd.DataFrame({'date': unique_trade_dates})

# filter for when Testing Starts
df_unique_trade_dates = df_unique_trade_dates[df_unique_trade_dates.date >= str(data_config.TESTING_START)]
df_unique_trade_dates.index = df_unique_trade_dates.date.factorize()[0]
df_unique_trade_dates

Found prevouisly saved pre-processed data: /Users/akeaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/data/csv/preprocessed_djia_data.csv


,date
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
1089,2020-05-01
1090,2020-05-04
1091,2020-05-05
1092,2020-05-06


In [27]:
df_training_results_account_value = get_training_results_account_value(df_unique_trade_dates)
# df_training_results_account_value.account_value.plot()
df_training_results_account_value

Sharpe Ratio: 125.48 %


,date,account_value
0,2016-01-04,1.000000e+06
1,2016-01-05,1.000057e+06
2,2016-01-06,9.981705e+05
3,2016-01-07,9.934172e+05
4,2016-01-08,9.912063e+05
...,...,...
1066,2020-03-30,1.458181e+06
1067,2020-03-31,1.458181e+06
1068,2020-04-01,1.458181e+06
1069,2020-04-02,1.458181e+06


In [28]:
# get daily return as relative difference in account value
df_training_results_account_value['daily_return'] = df_training_results_account_value.account_value.pct_change(1)

# compare dates with test date
df_training_results_account_value['Date'] = df_djia_out_of_sample['Date']
df_training_results_account_value = df_training_results_account_value[['date', 'Date', 'account_value', 'daily_return']]

df_training_results_account_value

,date,Date,account_value,daily_return
0,2016-01-04,2016-01-04,1.000000e+06,NaN
1,2016-01-05,2016-01-05,1.000057e+06,0.000057
2,2016-01-06,2016-01-06,9.981705e+05,-0.001886
3,2016-01-07,2016-01-07,9.934172e+05,-0.004762
4,2016-01-08,2016-01-08,9.912063e+05,-0.002226
...,...,...,...,...
1066,2020-03-30,2020-03-30,1.458181e+06,0.000000
1067,2020-03-31,2020-03-31,1.458181e+06,0.000000
1068,2020-04-01,2020-04-01,1.458181e+06,0.000000
1069,2020-04-02,2020-04-02,1.458181e+06,0.000000


In [29]:
series_training_results_account_value = get_daily_return_as_series(df_training_results_account_value)  # [0:1097]
series_training_results_account_value

Date
2016-01-04 00:00:00+00:00         NaN
2016-01-05 00:00:00+00:00    0.000057
2016-01-06 00:00:00+00:00   -0.001886
2016-01-07 00:00:00+00:00   -0.004762
2016-01-08 00:00:00+00:00   -0.002226
                               ...   
2020-03-30 00:00:00+00:00    0.000000
2020-03-31 00:00:00+00:00    0.000000
2020-04-01 00:00:00+00:00    0.000000
2020-04-02 00:00:00+00:00    0.000000
2020-04-03 00:00:00+00:00    0.000000
Length: 1071, dtype: float64

### Pyfolio

In [30]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    pyfolio.create_full_tear_sheet(returns=series_training_results_account_value, benchmark_rets=series_djia_out_of_sample, set_context=False)

Start date,2016-01-04
End date,2020-04-03
Total months,51
,Backtest
Annual return,9.281%
Cumulative returns,45.818%
Annual volatility,7.293%
Sharpe ratio,1.25
Calmar ratio,1.55
Stability,0.88
Max drawdown,-5.991%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,5.99,2017-03-01,2017-05-12,2017-10-16,164
1,5.24,2016-06-08,2016-06-27,2016-07-11,24
2,5.21,2019-03-18,2019-04-17,NaT,NaN
3,5.20,2016-08-11,2016-10-18,2016-11-25,77
4,4.31,2018-06-08,2018-06-27,2018-07-25,34


/Users/akeaveny/opt/anaconda3/envs/deep-rl/lib/python3.8/site-packages/pyfolio/tears.py:906: UserWarning: Passed returns do not overlap with anyinteresting times.
  warnings.warn('Passed returns do not overlap with any'
